# Mount Containers

In [ ]:
def mount_adls_by_extra_app(
    tenant_id: str,
    client_id: str,
    client_secret: str,
    mounting_folder_path: str,
    storage_account_name: str,
    container_name: str,
    replace_if_exists: bool = False,
    display_mounts: bool = False
) -> str:
    mounting_folder_path = mounting_folder_path[1:] if mounting_folder_path.startswith("/") else mounting_folder_path
    
    configs = {
        "fs.azure.account.auth.type": "OAuth",
        "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
        "fs.azure.account.oauth2.client.id": client_id,
        "fs.azure.account.oauth2.client.secret": client_secret,
        "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"
    }

    source_path = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/"
    mount_point = f"/mnt/{mounting_folder_path}"
    
    # Unmount the mount point if it already exists
    if any(
        mount.mountPoint == mount_point
        for mount in dbutils.fs.mounts()
    ):
        if not replace_if_exists:
            display(dbutils.fs.mounts())
            return mount_point
        dbutils.fs.unmount(mount_point)
    
    # Mount the storage account container
    dbutils.fs.mount(
        source=source_path,
        mount_point=mount_point,
        extra_configs=configs
    )

    if display_mounts:
        display(dbutils.fs.mounts())
    
    return mount_point

In [ ]:
# => # secrets/createScope
dbutils.secrets.listScopes()

In [ ]:
STORAGE_ACCOUNT_NAME = "eadbdldevweusa"
MOUNT_FOLDER = "/datahub"
MOUNT_CONTAINERS = [
    "bronze",
    "silver",
    "gold"
]

for container in MOUNT_CONTAINERS:
    mount_desination = f"{MOUNT_FOLDER}/{container}"
    mount_point = mount_adls_by_extra_app(
        tenant_id=tenant_id,
        client_id=client_id,
        client_secret=client_secret,
        mounting_folder_path=mount_desination,
        storage_account_name=STORAGE_ACCOUNT_NAME,
        container_name=container,
        replace_if_exists=True,
        display_mounts=False
    )
    print("mounted:", mount_point)

In [ ]:
PATH = "dbfs:/mnt/datahub/bronze/demo/circuits.csv"
df = spark.read.csv(PATH, header=True)
display(df)